In [1]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime, date
from tqdm import tqdm_notebook as tqdm
import os


from multiprocessing import Pool
from functools import partial
import ALL_STOCK_preprocess_function as func

In [2]:
data = pd.read_csv('D:\\庫存健診開發\\data\\processed\\TWSE_stock.csv', converters={'StockNo': str})
data['ts'] = pd.to_datetime(data['ts'])
fundamental = pd.read_csv('D:\\庫存健診開發\\data\\processed\\Ondate.csv', converters={'StockNo': str})
fundamental['On_Date'] = pd.to_datetime(fundamental['On_Date'])
eliminate = pd.read_csv('D:\\庫存健診開發\\data\\processed\\eliminate.csv', converters={'StockNo': str})
eliminate['Stop_date'], eliminate['Restart_date'] = pd.to_datetime(eliminate['Stop_date']), pd.to_datetime(eliminate['Restart_date'])

In [3]:
df_list = [group[1] for group in data.groupby(data['StockNo'])]
timedf = pd.DataFrame(data['ts'].unique(), columns=['ts'])
timedf = timedf.sort_values(by='ts')
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.FillMissingTime, timedf=timedf, ondate=fundamental, eliminate=eliminate), df_list), total=len(df_list)), 1):
                output_list.append(x)
                
df = pd.concat(output_list)
df

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,VWAP,eliminate
0,2007-07-02,1101,台泥,16.68,16.68,16.15,16.37,10222.0,381140.0,1202.8,37.286245,0
1,2007-07-03,1101,台泥,16.46,16.50,16.18,16.33,9831.0,367582.0,1199.6,37.390093,0
2,2007-07-04,1101,台泥,16.37,16.77,16.31,16.46,11124.0,420368.0,1209.2,37.789284,0
3,2007-07-05,1101,台泥,16.46,16.59,16.15,16.15,15043.0,559331.0,1186.7,37.182145,0
4,2007-07-06,1101,台泥,16.15,16.87,16.04,16.83,25514.0,959258.0,1236.5,37.597319,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2879,2019-09-17,9955,佳龍,19.50,19.60,19.30,19.30,544.0,10571.0,19.9,19.431985,0
2880,2019-09-18,9955,佳龍,19.30,19.35,19.15,19.15,442.0,8499.0,19.8,19.228507,0
2881,2019-09-19,9955,佳龍,19.10,19.10,18.65,18.80,957.0,18003.0,19.4,18.811912,0
2882,2019-09-20,9955,佳龍,18.80,19.20,18.80,19.15,538.0,10280.0,19.8,19.107807,0


In [4]:
index = pd.read_csv('D:\\庫存健診開發\\data\\processed\\index.csv')
industry_index = pd.read_csv('D:\\庫存健診開發\\data\\processed\\industry_index.csv')
index['ts'] = pd.to_datetime(index['ts'])
industry_index['ts'] = pd.to_datetime(industry_index['ts'])

In [5]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.merge_index, index=index, industry_index=industry_index, fundamental=fundamental), df_list), total=len(df_list)), 1):
                output_list.append(x)
                
df = pd.concat(output_list)
df

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,index_open,index_high,index_low,index_close,index_vol,industry_open,industry_high,industry_low,industry_close,industry_vol
0,2007-07-02,1101,台泥,16.68,16.68,16.15,16.37,10222.0,381140.0,1202.8,...,8903.77,8939.49,8849.35,8939.49,5490199.0,125.08,125.09,122.87,124.56,44467.0
1,2007-07-03,1101,台泥,16.46,16.50,16.18,16.33,9831.0,367582.0,1199.6,...,8983.24,9019.93,8977.09,8996.20,6604477.0,124.69,125.49,123.77,124.54,76827.0
2,2007-07-04,1101,台泥,16.37,16.77,16.31,16.46,11124.0,420368.0,1209.2,...,9069.32,9112.74,9034.50,9068.98,7261940.0,124.83,127.13,124.56,125.13,48887.0
3,2007-07-05,1101,台泥,16.46,16.59,16.15,16.15,15043.0,559331.0,1186.7,...,9114.06,9148.78,9086.19,9148.78,7065444.0,125.46,125.46,122.82,123.23,55899.0
4,2007-07-06,1101,台泥,16.15,16.87,16.04,16.83,25514.0,959258.0,1236.5,...,9169.56,9199.66,9118.26,9188.31,7514284.0,122.77,125.91,121.95,125.67,103863.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849,2019-09-17,9958,世紀鋼,78.59,78.59,76.80,77.00,3050.0,237191.0,161.8,...,10908.85,10912.23,10866.43,10874.50,4238581.0,102.59,102.71,101.54,101.55,31349.0
2850,2019-09-18,9958,世紀鋼,77.90,79.30,77.50,77.70,5940.0,464879.0,162.6,...,10887.42,10971.54,10878.34,10929.45,4380466.0,101.42,102.19,101.42,101.80,33530.0
2851,2019-09-19,9958,世紀鋼,78.40,78.40,76.20,76.30,4011.0,308696.0,159.7,...,10947.62,10954.18,10874.66,10894.70,4094710.0,101.80,102.03,101.31,101.59,22747.0
2852,2019-09-20,9958,世紀鋼,76.60,77.00,75.80,75.80,1742.0,132881.0,158.6,...,10908.10,10931.76,10903.45,10929.69,4712806.0,101.46,102.10,101.33,102.10,55930.0


In [8]:
inventory = pd.read_csv('D:\\庫存健診開發\\data\\processed\\inventory.csv', converters={'StockNo': str})
inventory['ts'] = pd.to_datetime(inventory['ts'])
inventory

,ts,StockNo,foreign_buy,investment_buy,dealer_buy
0,2004-02-10,0001,0.0,0.0,0
1,2004-02-10,5452,0.0,0.0,0
2,2004-02-10,5450,0.0,0.0,0
3,2004-02-10,5443,0.0,0.0,0
4,2004-02-10,5442R,0.0,0.0,0
...,...,...,...,...,...
6246018,2019-10-29,5258,-102.0,0.0,-2
6246019,2019-10-29,5259,0.0,0.0,0
6246020,2019-10-29,5263,35.0,-70.0,-11
6246021,2019-10-29,5213,-8.0,0.0,0


In [22]:
df_new = pd.merge(df, inventory, on=['ts', 'StockNo'], how='left')
df_new = df_new.fillna(0)
df_new

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,index_close,index_vol,industry_open,industry_high,industry_low,industry_close,industry_vol,foreign_buy,investment_buy,dealer_buy
0,2007-07-02,1101,台泥,16.68,16.68,16.15,16.37,10222.0,381140.0,1202.8,...,8939.49,5490199.0,125.08,125.09,122.87,124.56,44467.0,-3917.0,-765.0,158.0
1,2007-07-03,1101,台泥,16.46,16.50,16.18,16.33,9831.0,367582.0,1199.6,...,8996.20,6604477.0,124.69,125.49,123.77,124.54,76827.0,-1035.0,0.0,1255.0
2,2007-07-04,1101,台泥,16.37,16.77,16.31,16.46,11124.0,420368.0,1209.2,...,9068.98,7261940.0,124.83,127.13,124.56,125.13,48887.0,1057.0,680.0,198.0
3,2007-07-05,1101,台泥,16.46,16.59,16.15,16.15,15043.0,559331.0,1186.7,...,9148.78,7065444.0,125.46,125.46,122.82,123.23,55899.0,-8619.0,601.0,74.0
4,2007-07-06,1101,台泥,16.15,16.87,16.04,16.83,25514.0,959258.0,1236.5,...,9188.31,7514284.0,122.77,125.91,121.95,125.67,103863.0,5288.0,1485.0,1155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369242,2019-09-17,9958,世紀鋼,78.59,78.59,76.80,77.00,3050.0,237191.0,161.8,...,10874.50,4238581.0,102.59,102.71,101.54,101.55,31349.0,508.0,0.0,-358.0
2369243,2019-09-18,9958,世紀鋼,77.90,79.30,77.50,77.70,5940.0,464879.0,162.6,...,10929.45,4380466.0,101.42,102.19,101.42,101.80,33530.0,-291.0,0.0,144.0
2369244,2019-09-19,9958,世紀鋼,78.40,78.40,76.20,76.30,4011.0,308696.0,159.7,...,10894.70,4094710.0,101.80,102.03,101.31,101.59,22747.0,64.0,0.0,-161.0
2369245,2019-09-20,9958,世紀鋼,76.60,77.00,75.80,75.80,1742.0,132881.0,158.6,...,10929.69,4712806.0,101.46,102.10,101.33,102.10,55930.0,144.0,0.0,21.0


In [24]:
df_new.to_csv('D:\\庫存健診開發\\data\\processed\\TWSE_stock_processed.csv', index=False)

In [25]:
df_new.columns

Index(['ts', 'StockNo', 'StockName', 'open', 'high', 'low', 'close', 'vol',
       'total', 'capital', 'VWAP', 'eliminate', 'year', 'industry',
       'index_open', 'index_high', 'index_low', 'index_close', 'index_vol',
       'industry_open', 'industry_high', 'industry_low', 'industry_close',
       'industry_vol', 'foreign_buy', 'investment_buy', 'dealer_buy'],
      dtype='object')

In [27]:
df_new[df_new.industry_vol == 0]

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,index_close,index_vol,industry_open,industry_high,industry_low,industry_close,industry_vol,foreign_buy,investment_buy,dealer_buy


In [28]:
df_new[df_new.eliminate == 1]

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,index_close,index_vol,industry_open,industry_high,industry_low,industry_close,industry_vol,foreign_buy,investment_buy,dealer_buy
14159,2015-10-16,1109,信大,10.02,10.07,9.88,9.98,0.0,0.0,44.0,...,8604.95,4770762.0,125.96,127.23,125.71,126.74,9723.0,0.0,0.0,0.0
14652,2017-10-19,1109,信大,12.33,12.43,11.94,12.33,0.0,0.0,47.7,...,10760.29,5461983.0,111.90,111.90,111.41,111.51,6908.0,0.0,0.0,0.0
14893,2018-10-11,1109,信大,14.52,14.67,14.38,14.38,0.0,0.0,53.1,...,9806.11,8106979.0,142.03,142.03,137.34,138.54,59092.0,0.0,0.0,0.0
68993,2013-10-31,1234,黑松,31.20,31.48,30.74,30.84,0.0,0.0,180.6,...,8450.06,4012709.0,1282.22,1292.17,1275.70,1283.66,40475.0,0.0,0.0,0.0
103055,2017-01-12,1316,上曜,12.81,12.98,12.59,12.81,0.0,0.0,19.4,...,9410.18,3771167.0,99.98,100.31,99.81,100.01,23951.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309941,2017-01-03,9928,中視,9.61,9.95,9.42,9.61,0.0,0.0,6.8,...,9272.88,2490220.0,236.16,236.51,235.26,235.89,15418.0,0.0,0.0,0.0
2327655,2015-03-27,9929,秋雨,12.61,13.62,12.61,13.31,0.0,0.0,12.8,...,9503.72,5202683.0,278.01,279.63,276.95,278.30,59704.0,0.0,0.0,0.0
2332897,2011-12-15,9935,慶豐富,4.52,4.55,4.14,4.51,0.0,0.0,5.4,...,6764.59,2778077.0,139.35,139.35,137.87,138.15,25134.0,0.0,0.0,0.0
2355235,2010-09-14,9943,好樂迪,15.69,15.97,15.48,15.69,0.0,0.0,46.7,...,8132.60,4843343.0,158.02,158.40,157.41,157.69,80564.0,0.0,0.0,0.0


In [29]:
df_new[df_new.eliminate == 2]

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,index_close,index_vol,industry_open,industry_high,industry_low,industry_close,industry_vol,foreign_buy,investment_buy,dealer_buy
14169,2015-10-30,1109,信大,9.83,9.88,9.60,9.79,163.0,1671.0,39.0,...,8554.31,4366503.0,125.52,126.11,124.73,124.73,7111.0,-8.0,0.0,0.0
14659,2017-10-30,1109,信大,11.81,12.54,11.81,12.30,195.0,2448.0,45.7,...,10756.87,5553483.0,111.07,111.29,110.91,111.04,4884.0,40.0,0.0,0.0
14900,2018-10-22,1109,信大,13.61,13.99,13.51,13.75,423.0,6055.0,49.1,...,9974.28,4612359.0,134.96,137.28,134.87,136.00,22905.0,-1.0,0.0,0.0
69001,2013-11-12,1234,黑松,28.69,28.69,28.69,28.69,363.0,14078.0,155.5,...,8195.26,3832070.0,1201.48,1225.20,1191.36,1225.20,47613.0,-9.0,0.0,0.0
103063,2017-01-24,1316,上曜,14.05,14.05,13.37,14.05,2106.0,38537.0,21.2,...,9447.95,3362192.0,100.51,101.28,100.50,101.11,41128.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309950,2017-01-16,9928,中視,8.64,9.60,8.64,8.71,14.0,129.0,6.2,...,9292.33,3066710.0,237.88,238.26,236.93,237.55,23392.0,0.0,0.0,0.0
2327663,2015-04-10,9929,秋雨,13.65,13.65,13.65,13.65,13.0,182.0,13.1,...,9617.70,4888368.0,280.87,283.07,280.28,281.46,82518.0,0.0,0.0,0.0
2332909,2012-01-02,9935,慶豐富,4.82,4.82,4.27,4.82,441.0,2090.0,6.2,...,6952.21,2104641.0,142.38,142.41,140.37,140.58,17630.0,0.0,0.0,0.0
2355246,2010-09-30,9943,好樂迪,16.13,16.67,15.86,15.97,321.0,9661.0,43.7,...,8237.78,4519282.0,160.06,160.32,158.74,159.73,39806.0,-13.0,0.0,0.0
